In [6]:
import azureml.core
from azureml.core import Experiment, Environment, Workspace, Dataset, Datastore, ScriptRunConfig
import os
import shutil

# check core SDK version number

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.18.0


In [7]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
# get metadata about the workspace
print(ws.name, ws.location, ws.resource_group, sep='\t')
# list the registered datastores
ws.datastores

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EDABQYWG2 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
solar-compviz	eastus2	PLACE


{'solardatafilestore': {
   "name": "solardatafilestore",
   "container_name": "gee-data",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceblobstore': {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-20200d5f-4836-4a79-bf1c-7bd46e0d0c35",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-20200d5f-4836-4a79-bf1c-7bd46e0d0c35",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 }}

In [4]:
# register our data share containing image data in this workspace
Datastore.register_azure_file_share(
    account_key = 'andMlwgRrZ68ybpJ3d24F9pffr+tqPxlnLxM6qqB8gD75BvMrQKr/6IbvUpRy8cBPytynlLoc78iN2TFU7rWPA==',
    workspace = ws,
    datastore_name = 'solardatafilestore',
    file_share_name = 'gee-data',
    account_name = 'solarcompviz6161005396')

{
  "name": "solardatafilestore",
  "container_name": "gee-data",
  "account_name": "solarcompviz6161005396",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [5]:
# create a file dataset that can be used in training
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatafilestore')
datastore_paths = [(datastore, 'CPK_solar'), (datastore, 'NC_solar')]
cpk_dataset = Dataset.File.from_files(path = datastore_paths[0])
nc_dataset = Dataset.File.from_files(path = datastore_paths[1])

cpk_dataset = cpk_dataset.register(
    workspace=ws,
    name='gee-cpk-solar-data',
    description='training and eval TFRecords for solar arrays exported from GEE',
    create_new_version=True)

nc_dataset = nc_dataset.register(
    workspace=ws,
    name='gee-nc-solar-data',
    description='training and eval TFRecords for solar arrays exported from GEE',
    create_new_version=True)

In [8]:
# create an environment for the first time

envs = Environment.list(workspace = ws)
# well start with a pre-built tensorflow environment
env = envs.get('AzureML-TensorFlow-2.3-GPU')
env

# define packages to be installed using CondaDependencies
# get the packages that are already part of the pre-built environment
conda_dep = env.python.conda_dependencies
# list packages to install
pip_packages = ['matplotlib', 'rasterio', 'tensorboard']

# add each package to the existing conda dependencies
for package in pip_packages:
    conda_dep.add_pip_package(package)

# double check all the packages are there
conda_dep.serialize_to_string()
# conda_dep = CondaDependencies.create(
#     pip_packages=pip_packages)

# Now update the conda dependencies of the python environment
env.python.conda_dependencies=conda_dep

# # Register environment to re-use later
env.name = 'solar-training'
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04:20210513.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "solar-training",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
           